In [1]:
import os
import sys
import shutil
import random
import time
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
sys.path.append(os.path.join(PROJ_DIR, "src"))

import SharedArray as sa
import numpy as np
import time

from pkg.utils.utils import *

args = DummyObject()
args.model_path = "None"
args.rtype="panda"
args.precision="FP16"

if args.model_path == "None":
    args.model_path = None
PRECISION = args.precision

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2


from pkg.utils.utils_python3 import *
from pkg.controller.robot_config import RobotType
from pkg.planning.filtering.lattice_model.data_utils import *
import numpy as np
int2rtypename = {v.value:v.name for v in RobotType}
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

ARM_FOLDER = ARM10_FOLDER
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

BATCH_SIZE = 1
SERVER_PERIOD = 1e-3

## TensorRT version

In [2]:
##
# @class SharedLatticePredictor
class SharedLatticePredictor:
    ##
    # @param ROBOT_TYPE_NAME robot type name
    # @param model_path_rel relative model path from model/latticized/
    def __init__(self, ROBOT_TYPE_NAME="indy7", model_path_rel=None):
        self.ROBOT_TYPE_NAME = ROBOT_TYPE_NAME
        self.ROBOT_MODEL_ROOT = os.path.join(LAT_MODEL_PATH, self.ROBOT_TYPE_NAME)
        if model_path_rel is None:
            last_model = sorted(os.listdir(self.ROBOT_MODEL_ROOT))[-1]
            last_save = sorted([item for item in os.listdir(os.path.join(self.ROBOT_MODEL_ROOT, last_model)) if item.startswith("model")])[-1]
            model_path_rel = os.path.join(last_model, last_save)
        model_log_dir = os.path.join(self.ROBOT_MODEL_ROOT, model_path_rel)
        model_log_dir_trt = os.path.join(self.ROBOT_MODEL_ROOT, model_path_rel.replace("model", "trt")+"-"+PRECISION)
        if not os.path.isdir(model_log_dir_trt):
            print("==== Start converting ====")
            from tensorflow.python.compiler.tensorrt import trt_convert as trt

            conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
            conversion_params = conversion_params._replace(precision_mode=PRECISION) # Set GPU temporary memory 4GB
                
            converter = trt.TrtGraphConverterV2(input_saved_model_dir=model_log_dir, conversion_params=conversion_params)
            converter.convert()
                
            def my_input_fn():
                grasp_img_t = tf.zeros((BATCH_SIZE,) + GRASP_SHAPE + (3,), dtype=tf.float32)
                arm_img_t = tf.zeros((BATCH_SIZE,) + ARM_SHAPE + (1,), dtype=tf.float32)
                rh_mask_t = tf.zeros((BATCH_SIZE, 54), dtype=tf.float32)
                yield (grasp_img_t, arm_img_t, rh_mask_t)
                
            converter.build(input_fn=my_input_fn)
            print("==== Conversion Done ====")
            converter.save(model_log_dir_trt)
            print("==== Saved Converted model ====")

        saved_model_loaded = tf.saved_model.load(
            model_log_dir_trt, tags=[tag_constants.SERVING])
        graph_func = saved_model_loaded.signatures[
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
        self.frozen_func = convert_variables_to_constants_v2(graph_func)
        @tf.function
        def inference(inp1, inp2, inp3):
            predictions = self.frozen_func(inp1, inp2, inp3)[0]
            return predictions
        self.inference = lambda x: inference(*x).numpy()

    ##
    # @brief Create an array in shared memory.
    # @param prepared_p bool shared array (1,) to signal readiness
    def start_server(self, prepared_p):
        grasp_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
        arm_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
        rh_vals_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
        result_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
        query_in = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
        response_out = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
        query_quit = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
        gtimer = GlobalTimer.instance()
        gtimer.reset()
        grasp_img_p[:] = 0
        arm_img_p[:] = 0
        rh_vals_p[:] = 0
        result_p[:] = 0
        query_in[0] = False
        response_out[0] = False
        query_quit[0] = False
        rh_mask = np.zeros((BATCH_SIZE, 54))

        print("============= wait for initialization ================")
        r_mask = div_r_gaussian(rh_vals_p[0][0])
        h_mask = div_h_gaussian(rh_vals_p[0][1])
        rh_mask[0] = np.concatenate([r_mask, h_mask])
        grasp_img_t = tf.constant(grasp_img_p, dtype=tf.float32)
        arm_img_t = tf.constant(arm_img_p, dtype=tf.float32)
        rh_mask_t = tf.constant(rh_mask, dtype=tf.float32)
        self.inference((grasp_img_t, arm_img_t, rh_mask_t))
        print("=============== initialization done ==================")
        prepared_p[0] = True

        try:
            while not query_quit[0]:
                if not query_in[0]:
                    time.sleep(SERVER_PERIOD)
                    continue
                gtimer.tic("convert_tf")
                query_in[0] = False
                ## TODO: inference depending on robot type
                r_mask = div_r_gaussian(rh_vals_p[0][0])
                h_mask = div_h_gaussian(rh_vals_p[0][1])
                rh_mask[0] = np.concatenate([r_mask, h_mask])
                inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
                          tf.constant(arm_img_p, dtype=tf.float32), 
                          tf.constant(rh_mask, dtype=tf.float32)]
                etime_ctf = gtimer.toc("convert_tf")
                gtimer.tic("inference")
                result = self.inference((grasp_img_t, arm_img_t, rh_mask_t))
                for i_b in range(BATCH_SIZE):
                    result_p[i_b] = result[i_b]
                etime_inf = gtimer.toc("inference")
                print("convertin : {} ms".format(round(etime_ctf, 2)))
                print("inference : {} ms".format(round(etime_inf, 2)))
                response_out[0] = True
        finally:
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.arm_img")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.result")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_in")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.response_out")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_quit")


In [3]:
slp = SharedLatticePredictor(ROBOT_TYPE_NAME=args.rtype, model_path_rel=args.model_path)

In [5]:
self, prepared_p = slp, sa.create(f"shm://{args.rtype}.prepared", (1,), dtype=np.bool)

grasp_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
arm_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
rh_vals_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
result_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
query_in = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
response_out = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
query_quit = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
gtimer = GlobalTimer.instance()
gtimer.reset()
grasp_img_p[:] = 0
arm_img_p[:] = 0
rh_vals_p[:] = 0
result_p[:] = 0
query_in[0] = False
response_out[0] = False
query_quit[0] = False
rh_mask = np.zeros((BATCH_SIZE, 54))

print("============= wait for initialization ================")
r_mask = div_r_gaussian(rh_vals_p[0][0])
h_mask = div_h_gaussian(rh_vals_p[0][1])
rh_mask[0] = np.concatenate([r_mask, h_mask])
inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
          tf.constant(arm_img_p, dtype=tf.float32), 
          tf.constant(rh_mask, dtype=tf.float32)]
self.inference(inputs)
print("=============== initialization done ==================")
prepared_p[0] = True


============= wait for initialization ================
=============== initialization done ==================


In [7]:
gtimer.tic("convert_tf")
query_in[0] = False
## TODO: inference depending on robot type
r_mask = div_r_gaussian(rh_vals_p[0][0])
h_mask = div_h_gaussian(rh_vals_p[0][1])
rh_mask[0] = np.concatenate([r_mask, h_mask])
inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
          tf.constant(arm_img_p, dtype=tf.float32), 
          tf.constant(rh_mask, dtype=tf.float32)]
etime_ctf = gtimer.toc("convert_tf")
gtimer.tic("inference")
result = self.inference(inputs)
for i_b in range(BATCH_SIZE):
    result_p[i_b] = result[i_b]
etime_inf = gtimer.toc("inference")
print("convertin : {} ms".format(round(etime_ctf, 2)))
print("inference : {} ms".format(round(etime_inf, 2)))
response_out[0] = True

convertin : 3.95 ms
inference : 35.58 ms


In [8]:
while not query_quit[0]:
    if not query_in[0]:
        time.sleep(SERVER_PERIOD)
        continue
    gtimer.tic("convert_tf")
    query_in[0] = False
    ## TODO: inference depending on robot type
    r_mask = div_r_gaussian(rh_vals_p[0][0])
    h_mask = div_h_gaussian(rh_vals_p[0][1])
    rh_mask[0] = np.concatenate([r_mask, h_mask])
    inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
              tf.constant(arm_img_p, dtype=tf.float32), 
              tf.constant(rh_mask, dtype=tf.float32)]
    etime_ctf = gtimer.toc("convert_tf")
    gtimer.tic("inference")
    result = self.inference(inputs)
    for i_b in range(BATCH_SIZE):
        result_p[i_b] = result[i_b]
    etime_inf = gtimer.toc("inference")
    response_out[0] = True
query_quit[0] = False

In [9]:
print(gtimer)

convert_tf: 	189.0 ms/102 = 1.849 ms (0.453/15.803)
inference: 	1575.0 ms/102 = 15.446 ms (12.674/35.579)



In [8]:
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.arm_img")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.result")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_in")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.response_out")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_quit")
sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.prepared")

# Test Original

In [11]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


##
# @class SharedLatticePredictor
class SharedLatticePredictor:
    ##
    # @param ROBOT_TYPE_NAME robot type name
    # @param model_path_rel relative model path from model/latticized/
    def __init__(self, ROBOT_TYPE_NAME="indy7", model_path_rel=None):
        self.ROBOT_TYPE_NAME = ROBOT_TYPE_NAME
        self.ROBOT_MODEL_ROOT = os.path.join(LAT_MODEL_PATH, self.ROBOT_TYPE_NAME)
        if model_path_rel is None:
            last_model = sorted(os.listdir(self.ROBOT_MODEL_ROOT))[-1]
            last_save = sorted([item for item in os.listdir(os.path.join(self.ROBOT_MODEL_ROOT, last_model)) if item.startswith("model")])[-1]
            model_path_rel = os.path.join(last_model, last_save)
        model_log_dir = os.path.join(self.ROBOT_MODEL_ROOT, model_path_rel)
        self.model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(self, images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = self.model(images, training=False)
        return predictions

    ##
    # @brief Create an array in shared memory.
    # @param prepared_p bool shared array (1,) to signal readiness
    def start_server(self, prepared_p):
        grasp_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
        arm_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
        rh_vals_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
        result_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
        query_in = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
        response_out = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
        query_quit = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
        gtimer = GlobalTimer.instance()
        gtimer.reset()
        grasp_img_p[:] = 0
        arm_img_p[:] = 0
        rh_vals_p[:] = 0
        result_p[:] = 0
        query_in[0] = False
        response_out[0] = False
        query_quit[0] = False
        rh_mask = np.zeros((BATCH_SIZE, 54))

        print("============= wait for initialization ================")
        r_mask = div_r_gaussian(rh_vals_p[0][0])
        h_mask = div_h_gaussian(rh_vals_p[0][1])
        rh_mask[0] = np.concatenate([r_mask, h_mask])
        inputs = [tf.constant(grasp_img_p, dtype=tf.float32),
                  tf.constant(arm_img_p, dtype=tf.float32),
                  tf.constant(rh_mask, dtype=tf.float32)]
        self.inference(inputs)
        print("=============== initialization done ==================")
        prepared_p[0] = True

        try:
            while not query_quit[0]:
                if not query_in[0]:
                    time.sleep(SERVER_PERIOD)
                    continue
                query_in[0] = False
                ## TODO: inference depending on robot type
                r_mask = div_r_gaussian(rh_vals_p[0][0])
                h_mask = div_h_gaussian(rh_vals_p[0][1])
                rh_mask[0] = np.concatenate([r_mask, h_mask])
                inputs = [tf.constant(grasp_img_p, dtype=tf.float32),
                          tf.constant(arm_img_p, dtype=tf.float32),
                          tf.constant(rh_mask, dtype=tf.float32)]
                result = self.inference(inputs)
                for i_b in range(BATCH_SIZE):
                    result_p[i_b] = result[i_b]
                response_out[0] = True
        finally:
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.arm_img")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.result")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_in")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.response_out")
            sa.delete(f"shm://{self.ROBOT_TYPE_NAME}.query_quit")


/home/rnb/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1766: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [12]:
slp = SharedLatticePredictor(ROBOT_TYPE_NAME=args.rtype, model_path_rel=args.model_path)

In [15]:
self = slp

grasp_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
arm_img_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
rh_vals_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
result_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
query_in = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
response_out = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
query_quit = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
prepared_p = sa.create(f"shm://{self.ROBOT_TYPE_NAME}.prepared", (1,), dtype=np.bool)
prepared_p[0] = False

grasp_img_p[:] = 0
arm_img_p[:] = 0
rh_vals_p[:] = 0
result_p[:] = 0
query_in[0] = False
response_out[0] = False
query_quit[0] = False
rh_mask = np.zeros((BATCH_SIZE, 54))

print("============= wait for initialization ================")
r_mask = div_r_gaussian(rh_vals_p[0][0])
h_mask = div_h_gaussian(rh_vals_p[0][1])
rh_mask[0] = np.concatenate([r_mask, h_mask])
inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
          tf.constant(arm_img_p, dtype=tf.float32), 
          tf.constant(rh_mask, dtype=tf.float32)]
self.inference(inputs)
print("=============== initialization done ==================")
prepared_p[0] = True

============= wait for initialization ================
=============== initialization done ==================


### Call checker once to get data example. run below cell to return response so the checker can stop waiting

In [17]:
gtimer=GlobalTimer.instance()
gtimer.reset()
with gtimer.block("convert_tf"):
    query_in[0] = False
    ## TODO: inference depending on robot type
    r_mask = div_r_gaussian(rh_vals_p[0][0])
    h_mask = div_h_gaussian(rh_vals_p[0][1])
    rh_mask[0] = np.concatenate([r_mask, h_mask])
inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
          tf.constant(arm_img_p, dtype=tf.float32), 
          tf.constant(rh_mask, dtype=tf.float32)]
with gtimer.block("inference"):
    result = self.inference(inputs)
for i_b in range(BATCH_SIZE):
    result_p[i_b] = result[i_b]
response_out[0] = True
print(gtimer)

convert_tf: 	0.0 ms/1 = 0.061 ms (0.061/0.061)
inference: 	12.0 ms/1 = 12.208 ms (12.208/12.208)



In [18]:
gtimer.reset()
while not query_quit[0]:
    if not query_in[0]:
        time.sleep(SERVER_PERIOD)
        continue
    gtimer.tic("convert_tf")
    query_in[0] = False
    ## TODO: inference depending on robot type
    r_mask = div_r_gaussian(rh_vals_p[0][0])
    h_mask = div_h_gaussian(rh_vals_p[0][1])
    rh_mask[0] = np.concatenate([r_mask, h_mask])
    inputs = [tf.constant(grasp_img_p, dtype=tf.float32), 
              tf.constant(arm_img_p, dtype=tf.float32), 
              tf.constant(rh_mask, dtype=tf.float32)]
    etime_ctf = gtimer.toc("convert_tf")
    gtimer.tic("inference")
    result = self.inference(inputs).numpy()
    etime_inf = gtimer.toc("inference")
    gtimer.tic("set_response")
    for i_b in range(BATCH_SIZE):
        result_p[i_b] = result[i_b]
    response_out[0] = True
    etime_inf = gtimer.toc("set_response")
query_quit[0]=False
print(gtimer)

convert_tf: 	92.0 ms/101 = 0.91 ms (0.456/17.28)
inference: 	1613.0 ms/101 = 15.967 ms (15.226/17.097)
set_response: 	1.0 ms/101 = 0.007 ms (0.005/0.021)



## Mulst call below to clear shared memory

In [2]:
ROBOT_TYPE_NAME = "panda"
sa.delete(f"shm://{ROBOT_TYPE_NAME}.grasp_img")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.arm_img")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.rh_vals")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.result")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.query_in")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.response_out")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.query_quit")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.prepared")